# Transform the data into a complete time series

In [1]:
import pandas as pd
from src.paths import CLEANED_DATA, TRANSFORMED_DATA, GEOGRAPHICAL_DATA
from src.data_transformations import add_missing_slots
from src.miscellaneous import add_rounded_coordinates_to_dataframe, add_column_of_rounded_points, make_new_station_ids,save_dict, add_column_of_ids 

## Import the cleaned 2023 data

In [2]:
trips = pd.read_parquet(path = CLEANED_DATA/"final.parquet")
trips.head()

,start_time,stop_time,start_latitude,start_longitude,stop_latitude,stop_longitude
0,2023-01-21 20:05:42,2023-01-21 20:16:33,41.924074,-87.646278,41.930000,-87.640000
1,2023-01-10 15:37:36,2023-01-10 15:46:05,41.799568,-87.594747,41.809835,-87.599383
2,2023-01-02 07:51:57,2023-01-02 08:05:11,42.008571,-87.690483,42.039742,-87.699413
3,2023-01-22 10:52:58,2023-01-22 11:01:44,41.799568,-87.594747,41.809835,-87.599383
4,2023-01-12 13:58:01,2023-01-12 14:13:20,41.799568,-87.594747,41.809835,-87.599383


## Reduce the time stamps to hour-based values

In [3]:
trips["start_hour"] = trips["start_time"].dt.floor("H")
trips["stop_hour"] = trips["stop_time"].dt.floor("H")

trips = trips.drop(columns = ["start_time", "stop_time"])

## Preparing data for aggregation

### Starts 

In [4]:
starts = trips[
    [
        "start_hour", "start_latitude", "start_longitude"
    ]
] 


stops = trips[
    [
        "stop_hour", "stop_latitude", "stop_longitude"
    ]
]

##### Save original coordinates

In [5]:
starts[["start_latitude", "start_longitude"]].to_parquet(path = GEOGRAPHICAL_DATA/ "original_start_coordinates.parquet") 

stops[["stop_latitude", "stop_longitude"]].to_parquet(path = GEOGRAPHICAL_DATA/ "original_stop_coordinates.parquet") 

##### Round the coordinates to two decimal places to make grouping easier, and remove the old latitude and 

In [6]:
add_rounded_coordinates_to_dataframe(data = starts, decimal_places = 3, start_or_stop = "start")

 13%|█▎        | 710872/5495778 [00:04<00:31, 154236.70it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/kobina/Desktop/ML/End-to-End Projects/Actualised/cyclistic-bike-sharing-data/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_15038/963484252.py", line 1, in <module>
    add_rounded_coordinates_to_dataframe(data = starts, decimal_places = 3, start_or_stop = "start")
  File "/home/kobina/Desktop/ML/End-to-End Projects/Actualised/cyclistic-bike-sharing-data/src/miscellaneous.py", line 50, in add_rounded_coordinates_to_dataframe
    for latitude in tqdm(data[f"{start_or_stop}_latitude"].values):
  File "/home/kobina/Desktop/ML/End-to-End Projects/Actualised/cyclistic-bike-sharing-data/.venv/lib/python3.11/site-packages/tqdm/std.py", line -1, in __iter__
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kobina/Desktop/ML/End-to-End Proj

In [ ]:
add_rounded_coordinates_to_dataframe(data = stops, decimal_places = 3, start_or_stop = "stop")

100%|██████████| 5495778/5495778 [00:29<00:00, 183652.14it/s]


##### Remove original coordinates, and put those tuples in a dedicated column

In [ ]:
starts = starts.drop(columns = ["start_latitude", "start_longitude"])
stops = stops.drop(columns = ["stop_latitude", "stop_longitude"])

##### Add columns of consisting of tuples of rounded coordinates

In [ ]:
add_column_of_rounded_points(data = starts, start_or_stop = "start")
add_column_of_rounded_points(data = stops, start_or_stop = "stop")

In [10]:
starts = starts.drop(columns = ["rounded_start_latitude", "rounded_start_longitude"])
stops = stops.drop(columns = ["rounded_stop_latitude", "rounded_stop_longitude"])

#### Make new location IDs, and associate each of them to a point using a dictionary

In [ ]:
places_and_ids = make_new_station_ids(start_df = starts, stop_df = stops)

##### Ensuring that any points common to the origins and destinations have the same IDs

###### We have made dictionaries associating the origins and destinations with IDs, but some of the destinations and origins may be the same. And they will have been assigned to different IDs. These common locations (or rather, their coordinates) must be assigned to the same ID in each dictionary.


###### First, let us find out how many of these points are common to these dictionaries.

In [ ]:
common_points = [point for point in destinations_and_ids.keys() if point in origins_and_ids.keys()]
len(common_points)

###### There are 2,357 common locations. And they will most likely have been assigned to different IDs in each dictionary.
###### Let us ensure that these common points have the same IDs in each dictionary.

In [ ]:
for point in common_points:

        destinations_and_ids[point] = origins_and_ids[point]

###### Checking for repetition of origin IDs. The presence of repeated values necessitates a deeper investigation into whether they are shared by two different points.

In [ ]:
len(origins_and_ids.values()) == len(set(origins_and_ids.values()))

###### There are no repeated origin IDs 

In [ ]:
len(destinations_and_ids.values()) == len(set(destinations_and_ids.values()))

###### There are some repeated destination IDs. Let us check whether they belong to the same points or not. If they belong to two different points, then that will have to be rectified.

In [ ]:
for a,b in zip(destinations_and_ids.keys(), destinations_and_ids.keys()):

    if destinations_and_ids[a] == destinations_and_ids[b] and a != b:

        print((a,b))

###### All clear!

In [ ]:
stops

##### Save these dictionaries

###### This is crucial because it will allow me to recall this particular association of coordinates with IDs later on.

In [ ]:
save_dict(
    folder = GEOGRAPHICAL_DATA, 
    dictionary = origins_and_ids, 
    file_name = "rounded_origin_points_and_their_IDs.pkl"
    )

save_dict(
    folder = GEOGRAPHICAL_DATA, 
    dictionary = destinations_and_ids, 
    file_name = "rounded_destination_points_and_their_IDs.pkl"
    )

##### Form a column of said IDs (in the appropriate order)

In [ ]:
add_column_of_ids(data = starts, start_or_stop = "start", points_and_ids = origins_and_ids)

In [ ]:
add_column_of_ids(data = stops, start_or_stop = "stop", points_and_ids = destinations_and_ids)

## Form Aggregate Datasets

In [ ]:
starts = starts.drop("rounded_start_points", axis = 1)
stops = stops.drop("rounded_stop_points", axis = 1)

### Aggregate Starts

In [ ]:
agg_starts = starts.groupby(["start_hour", "start_station_id"]).size().reset_index()
agg_starts = agg_starts.rename(columns = {0: "trips"}) 

#### Full Aggregate Starts with Missing Slots

In [ ]:
ts_starts = add_missing_slots(agg_data = agg_starts, start_or_stop = "start")

In [ ]:
ts_starts.to_parquet(path = TRANSFORMED_DATA/"ts_starts.parquet")

### Aggregate Stops

In [ ]:
agg_stops = stops.groupby(["stop_hour", "stop_station_id"]).size().reset_index()
agg_stops = agg_stops.rename(columns = {0: "trips"}) 

#### Full Aggregate Stops with Missing Slots

In [ ]:
ts_stops = add_missing_slots(agg_data = agg_stops, start_or_stop = "stop")

In [ ]:
ts_stops.to_parquet(path = TRANSFORMED_DATA/"ts_stops.parquet")

In [ ]:
for i,j in zip([1,2], [3,4]):
    
    print(i,j)

1 3
2 4
